In [ ]:
from symbol import decorator

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.io as pio
from math import sqrt
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader, Dataset
import wandb
from models import FieldMLP
from utils import *

pio.renderers.default = "browser"

In [45]:
circle_sdf = CircleSDF(x0=[0,0], r=0.2)
circle_sdf.plot_field(True)

In [24]:
random = SDF(model=lambda x: torch.asarray(np.random.random((x.shape[0], 1))), xy_lims=(-1, 1, -1, 1))
random.plot_field(True, 5)

In [18]:
model = FieldMLP(hidden=128, depth=5).to(device)
sdf = SDF(model=model, xy_lims=(-1, 1, -1, 1))
wandb_logger = WandbLogger(project="ginn2d", name="run1")
lit = ConstraintTrainer(sdf, constraints=example_constraints, lr=1e-3, n_points=128, domain=[-1,1], seed=42)
trainer = pl.Trainer(
    max_epochs=50,
    accelerator="auto",
    precision="bf16-mixed" if torch.cuda.is_available() else "32-true",
    logger=wandb_logger,
    gradient_clip_val=1.0,
    log_every_n_steps=10,
)
trainer.fit(lit)

Seed set to 42
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\navra\anaconda3\envs\GINN\lib\site-packages\pytorch_lightning\loggers\wandb.py:396: UserWarning:

There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type     | Params | Mode 
-------------------------------------------
0 | model | FieldMLP | 50.0 K | train
-------------------------------------------
50.0 K    Trainable params
0         Non-trainable params
50.0 K    Total params
0.200     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode
C:\Users\navra\anaconda3\envs\GINN\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: PossibleUser

Epoch 49: 100%|██████████| 10/10 [00:00<00:00, 49.97it/s, v_num=xjob, loss_step=2.430, area_step=0.0531, tv_step=8.850, cont_step=0.094, center_step=1.960, loss_epoch=2.520, area_epoch=0.0606, tv_epoch=9.800, cont_epoch=0.097, center_epoch=1.640]  

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 10/10 [00:00<00:00, 45.44it/s, v_num=xjob, loss_step=2.430, area_step=0.0531, tv_step=8.850, cont_step=0.094, center_step=1.960, loss_epoch=2.520, area_epoch=0.0606, tv_epoch=9.800, cont_epoch=0.097, center_epoch=1.640]


In [6]:
metrics = trainer.logged_metrics
print(metrics)

{'loss_step': tensor(2.4623), 'area_step': tensor(0.0606), 'tv_step': tensor(9.5697), 'cont_step': tensor(0.0988), 'center_step': tensor(1.3733), 'loss_epoch': tensor(2.4568), 'area_epoch': tensor(0.0609), 'tv_epoch': tensor(9.3366), 'cont_epoch': tensor(0.0957), 'center_epoch': tensor(1.6123)}


In [7]:
sdf.plot_field(
    True, 50, 
)